In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

# 포트폴리오 구성
tickers = {
    'Nasdaq': '^IXIC',
    'Portfolio A': ['AAPL', 'MSFT', 'GOOGL'],
    'Portfolio B': ['JNJ', 'PFE', 'MRK'],
    'Portfolio C': ['JPM', 'BAC', 'C']
}

# 주가 데이터 불러오기
nasdaq = yf.download(tickers['Nasdaq'], start='2020-01-01', end='2023-12-31')['Adj Close']
portfolio_a = yf.download(tickers['Portfolio A'], start='2020-01-01', end='2023-12-31')['Adj Close']
portfolio_b = yf.download(tickers['Portfolio B'], start='2020-01-01', end='2023-12-31')['Adj Close']
portfolio_c = yf.download(tickers['Portfolio C'], start='2020-01-01', end='2023-12-31')['Adj Close']

# 각 포트폴리오의 평균 가격 계산
portfolio_a['Average'] = portfolio_a.mean(axis=1)
portfolio_b['Average'] = portfolio_b.mean(axis=1)
portfolio_c['Average'] = portfolio_c.mean(axis=1)

# 전체 데이터 합치기
prices = pd.DataFrame({
    'Nasdaq': nasdaq,
    'Portfolio A': portfolio_a['Average'],
    'Portfolio B': portfolio_b['Average'],
    'Portfolio C': portfolio_c['Average']
})

# 수익률 계산
returns = prices.pct_change().dropna()

# 전체기간 수익률
total_return = (prices.iloc[-1] / prices.iloc[0] - 1) * 100

# MDD 계산 함수
def calculate_mdd(prices):
    roll_max = prices.cummax()
    daily_drawdown = prices / roll_max - 1.0
    max_daily_drawdown = daily_drawdown.cummin()
    return max_daily_drawdown.min() * 100

# MDD 계산
mdd = prices.apply(calculate_mdd)

# 변동성 계산
volatility = returns.std() * np.sqrt(252) * 100

# 베타 계산
cov_matrix = returns.cov()
beta = cov_matrix.loc['Nasdaq', ['Portfolio A', 'Portfolio B', 'Portfolio C']] / returns['Nasdaq'].var()

# 샤프지수 계산
risk_free_rate = 0.01
sharpe_ratio = (returns.mean() * 252 - risk_free_rate) / (returns.std() * np.sqrt(252))

# 트래킹에러 계산
tracking_error = returns.sub(returns['Nasdaq'], axis=0).std() * np.sqrt(252)

# 정보비율 계산
information_ratio = (returns.sub(returns['Nasdaq'], axis=0).mean() * 252) / tracking_error

# 각 기간별 수익률 계산 함수
def calculate_period_return(prices, period):
    if len(prices) >= period:
        return (prices.iloc[-1] / prices.iloc[-period] - 1) * 100
    else:
        return np.nan

# 기간별 수익률 계산
periods = [20, 60, 120, 252, 504]  # 1개월, 3개월, 6개월, 12개월, 24개월
labels = ['1개월', '3개월', '6개월', '12개월', '24개월']

period_returns = {}
for label, period in zip(labels, periods):
    period_return = calculate_period_return(prices, period)
    period_returns[label] = period_return

period_returns_df = pd.DataFrame(period_returns)

# 결과를 데이터프레임으로 정리
result = pd.DataFrame({
    'Nasdaq': total_return['Nasdaq'],
    'Portfolio A': total_return['Portfolio A'],
    'Portfolio B': total_return['Portfolio B'],
    'Portfolio C': total_return['Portfolio C']
}, index=['전체기간 수익률'])

# 기간별 수익률 추가
for label in labels:
    result.loc[label] = period_returns_df[label]

# 나스닥 관련 없는 메트릭스는 빈 값으로 설정
additional_metrics = pd.DataFrame({
    'Nasdaq': [''] * 5,
    'Portfolio A': [volatility['Portfolio A'], beta['Portfolio A'], sharpe_ratio['Portfolio A'], tracking_error['Portfolio A'], information_ratio['Portfolio A']],
    'Portfolio B': [volatility['Portfolio B'], beta['Portfolio B'], sharpe_ratio['Portfolio B'], tracking_error['Portfolio B'], information_ratio['Portfolio B']],
    'Portfolio C': [volatility['Portfolio C'], beta['Portfolio C'], sharpe_ratio['Portfolio C'], tracking_error['Portfolio C'], information_ratio['Portfolio C']],
}, index=['변동성', '베타', '샤프지수', '트래킹에러', '정보비율'])

result = pd.concat([result, additional_metrics])

result.loc['MDD'] = [mdd['Nasdaq'], mdd['Portfolio A'], mdd['Portfolio B'], mdd['Portfolio C']]

# 결과 출력
result


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed


,Nasdaq,Portfolio A,Portfolio B,Portfolio C
전체기간 수익률,65.101575,138.967330,22.649674,12.471565
1개월,4.937559,1.528487,1.574982,8.586190
3개월,13.551759,12.691474,0.980318,22.628068
6개월,7.848213,9.979106,-0.720951,14.863705
12개월,43.264181,55.241039,-10.457334,24.680729
24개월,-4.787895,7.337148,2.403577,3.229198
변동성,,30.494457,19.510507,35.678681
베타,,1.054439,0.337783,0.790465
샤프지수,,0.836455,0.308369,0.232607
트래킹에러,,0.109841,0.248482,0.291377
